# Generation of synthetic promoters for *C. cinerea* from upstream sequences
## Purpose
This notebook extracts upstream sequences from the *C. cinerea* genome and trains a neural generator. It then generates synthetic promoters based on this generator.
## Extraction of upstream sequences

In [ ]:
#load required libraries
import pandas as pd
from Bio import SeqIO, AlignIO
import shutil
import subprocess
import os
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os